# In this data Notebook will be shown how the database for the paper.... has been built and how to include temporal information to static databases

In [1]:
import sys
# sys.path.insert(0, "/media/giuseppec/25F62A4E5FEED162/work/temporalis_update/brightway2-temporalis_last")
#use this one below until "biogenic_carbon" module (used in Wood_Decay below) is not updated
sys.path.insert(0, "/media/giuseppec/25F62A4E5FEED162/work/temporalis_update/brightway2-temporalis_tests")


In [2]:
from brightway2 import *
from bw2temporalis import *
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import copy
%matplotlib inline



In [3]:
projects.set_current('temporalis_demo')

In [4]:
# databases.list

we will be create a db with relative years and we set here the (arbitrary) year 0 as the year of haverst. Which we take as year 0 is actually little relevant since we will be run the analysis  with absoulte dates, it is just important to set correct relative timesteps between the unit processes with temporal informations

In [5]:
yr_harv=0
new_db_name='tempo'


 first we need to find the code of the unit process we want to include in the background the temporal data i.e `beam, softwood, raw, air drying to u=20%`

In [6]:
db=Database('ecoinvent 3.2 cutoff')

[[x['name'],x['reference product'],x['location'],x['code'],x['activity type'],x['activity'],x['filename']] 
 for x in db.search('"beam, softwood, raw, air drying to u=20%"', #activity name
                    filter = {'location':'ch'}, # CH not working
                    proxy=True)] 

# [[x['name'],x['reference product'],x['location'],x['code'],x['activity type'],x['activity'],x['filename']] 
#  for x in db.search('"sawnwood, beam, softwood, raw, dried (u=20%)"',#reference product name
#                     filter = {'location':'ch'}, # CH not working
#                     proxy=True)]

# db.search('"beam, softwood, raw, air drying to u=20%"', #activity name
#                     filter = {'location':'ch'}, # CH not working
#                     proxy=False)


Excluding 2 filtered results


[['beam, softwood, raw, air drying to u=20%',
  'sawnwood, beam, softwood, raw, dried (u=20%)',
  'CH',
  '12fabb4b0a9fc33d6f0812186295d724',
  'ordinary transforming activity',
  'd924d7d0-f579-44b5-92d0-69e61443db2d',
  'd924d7d0-f579-44b5-92d0-69e61443db2d_a1a4300c-8440-4eed-8205-24042aad6df7.spold']]

Now we look into its exchanges and we just go back untill we find the one with the forest management interventions. We do it manually with the function below and we go upstream the supply chain intil we find the unit process relative to forest management. This means that iteratively we look into the exchanges of the unit process and change the argument of `db.get` with the unit process we want to explore upstrea. For this can be of help the use of the GUI [Activity-browser](https://docs.brightwaylca.org/advanced-installation.html#activity-browser)

In [7]:
a=db.get('12fabb4b0a9fc33d6f0812186295d724')

print([a['name'],a['reference product'],a['location'],a['code']],':\n')
for exc in a.exchanges():
    if exc['type']=='biosphere':
        print([exc['name'],exc['type'],exc['input']])

['beam, softwood, raw, air drying to u=20%', 'sawnwood, beam, softwood, raw, dried (u=20%)', 'CH', '12fabb4b0a9fc33d6f0812186295d724'] :

['Transformation, to industrial area', 'biosphere', ('biosphere3', '4624deff-2016-41d4-b2bf-3db8dab88779')]
['Water', 'biosphere', ('biosphere3', '075e433b-4be4-448e-9510-9a5029c1ce94')]
['Transformation, from unspecified', 'biosphere', ('biosphere3', '29630a65-f38c-48a5-9744-c0121f586640')]
['Occupation, industrial area', 'biosphere', ('biosphere3', 'fe9c3a98-a6d2-452d-a9a4-a13e64f1b95b')]


With the previous step we can obtain the path until `softwood forestry, mixed species, sustainable forest management` that is the unit process we want to modify. 

P.S: we used a dictioanry below just to make more explicity what each code is.

In [8]:
#the keys are the activities, commented out the reference products. 

# THE FIRST HAS TO BE CHECKED!!

sawn={'beam, softwood, raw, air drying to u=20%': #'sawnwood, beam, softwood, raw, dried (u=20%)', 'CH',
        ('ecoinvent 3.2 cutoff', '12fabb4b0a9fc33d6f0812186295d724')}

sawn_upstream={
'sawing, softwood': #'sawnwood, softwood, raw', 'CH',
        ('ecoinvent 3.2 cutoff', 'abb367d014840ef43a947fbd2b75ab5b'),
'market for sawlog and veneer log, softwood, measured as solid wood under bark': # 'sawlog and veneer log, softwood, measured as solid wood under bark'
        ('ecoinvent 3.2 cutoff', '41ba0faa3437ad2c2fe52b6b8c00857c'),
'softwood forestry, mixed species, sustainable forest management': # 'sawlog and veneer log, softwood, measured as solid wood under bark', 'CH'
        ('ecoinvent 3.2 cutoff', 'd63bbaea0bba23c6b9f02b946d1e7bf8'),
}
# forest={'softwood forestry, mixed species, sustainable forest management': # 'sawlog and veneer log, softwood, measured as solid wood under bark', 'CH'
#         ('ecoinvent 3.2 cutoff','d63bbaea0bba23c6b9f02b946d1e7bf8')}

Now with the function below we start from `beam, softwood, raw, air drying to u=20%`  copy and all its exchanges as they are unless they are in the dictioanry `sawn_upstream`. In this case we apply the same approch iteratively untill we do not get to the `softwood forestry, mixed species, sustainable forest management`


In [9]:
def change_sawn(activity_code):
    old_db_name='ecoinvent 3.2 cutoff'
    activity=Database(old_db_name).get(activity_code)
    new_act=copy.deepcopy(activity.as_dict())
    new_act['database']=new_db_name
    new_act['exchanges']=[] #empty list of exchanges to add add with for loop below
    new_act_key=(new_db_name,new_act['code']) #we just keep same code
#     new_act_key=(new_db_name,new_act['name']) #we put name for readibility

    down=None
    
    for exc in activity.exchanges():
        if exc['input'] not in sawn_upstream.values() or exc['type']=='production':
            data = copy.deepcopy(exc._data)
            data['output'] = new_act_key

            # Change `input` for production exchanges
            if exc['input'] == exc['output']:
                data['input'] = new_act_key        
            new_act['exchanges'].append(data)
        else:
            data = copy.deepcopy(exc._data)
            data['output'] = new_act_key
            #need to change the input 
            data['input'] = (new_db_name,exc['input'][1]) #if  exc['input'] not in forest.values() else exc['input']
#             data['input'] = (new_db_name,exc['name']) #if  exc['input'] not in forest.values() else exc['input']

            #add two years delay between harvest and sawnwood productin
            if exc['input']==('ecoinvent 3.2 cutoff', '41ba0faa3437ad2c2fe52b6b8c00857c'):
                data['temporal distribution']=[(-2,exc['amount'])] #remember python is 0 indexed so range is up to -1             

            new_act['exchanges'].append(data)
            down=data['input']
            
    return new_act,down


new_db={}
key=sawn['beam, softwood, raw, air drying to u=20%']
while key != None:
    ds,key=change_sawn(key[1])
    
    new_db[(ds['database'],ds['code'])]=ds

 ok, we have copied our the upstram exch now we just take all the tecnosphere exchanges of the copied `softwood forestry, mixed species, sustainable forest management` and spread them over the 100 years of rotation.

In [10]:
for i,exc in enumerate(new_db[('tempo', 'd63bbaea0bba23c6b9f02b946d1e7bf8')]['exchanges']):
    if exc['type']=='technosphere':
        new_db[('tempo', 'd63bbaea0bba23c6b9f02b946d1e7bf8')]['exchanges'][i]['temporal distribution']=[(yr,exc['amount']/10) for yr in range(-90,1,10)] #remember python is 0 indexed so range is up to -1


Glulam production and its EOL were modelleded separately in Simapro and imported in a separate Bw2 database. Now we will take production and link it to the modified sawnwood with the relative temporal information (i.e. 2 years between sawnood production and glulam). First we search for the dataset with glulam production and EOL. Then we search for the sawnwood exchange in the production dataset, modifies it with the one created above and add this modified dataset and  he copy of the EOL dataset to the database we are creating

In [11]:
fur=Database('glu')
[[x['name'],x['code']] for x in fur.search('glulam')]

[['_softwood, primary construction, glulam beam, EOL_wood/ EU U',
  '99054f40ae985b96bbdfde3f0043d566'],
 ['_softwood, primary construction, glulam beam, production_wood/ EU U',
  'e2997759fbd9f7e785472c83d56a1db3'],
 ['_Softwood, Primary construction, glulam beam, Equivalence_EOL/ EU U',
  '8bd90a5f050dc85eadc6af6d98d02e59'],
 ['_Softwood, Primary construction, glulam beam, Equivalence_production/ EU U',
  '61ed29b7dab050397e6dcd5367f6f406']]

In [12]:
glu=fur.get('e2997759fbd9f7e785472c83d56a1db3') #get glulam prod
glu_eol=fur.get('99054f40ae985b96bbdfde3f0043d566') #get flulam eol

# [[x['name'],x['amount'],x['unit'],x['input']] for x in glu.exchanges()] #look into exchanges of eol

In [13]:
# for prod
new_glu_code='glulam_prod'

new_glu=copy.deepcopy(glu.as_dict())
new_glu['database']=new_db_name
new_glu['code']=new_glu_code
new_glu['name']=new_glu_code
new_glu['exchanges']=[] #empty list of exchanges to add add with for loop below
new_glu_key=(new_db_name,new_glu['code']) #we jsut keep same code and change db for semplicity

to_replace=(new_db_name,'12fabb4b0a9fc33d6f0812186295d724')

for exc in glu.exchanges():        
    data = copy.deepcopy(exc._data)
    data['output'] = new_glu_key
    # Change `input` for production exchanges
    if exc['input'] == exc['output']:
        data['input'] = new_glu_key
    #change sawnwood
    if exc['name']=='_Sawn timber, softwood, raw, kiln dried, u=20%, at plant':
        data['input']=to_replace
        data['name']=new_db[to_replace]['name']
        data['temporal distribution']=[(-2,exc['amount'])]  # sawnwood is produced 2 years before being used by glulam 
    new_glu['exchanges'].append(data)
new_db[new_glu_key]=new_glu
# new_glu['exchanges']


# test to add loop (to del later)

In [14]:
# new_db[('tempo', 'd63bbaea0bba23c6b9f02b946d1e7bf8')]['exchanges'].append({'amount': 0.9,
#   'input': new_glu_key,
#   'name': new_glu_code,
#   'output': ('tempo', 'd63bbaea0bba23c6b9f02b946d1e7bf8'),
#   'type': 'technosphere'})

In [15]:
#for eol
new_glu_eol_code='glulam_eol'

new_glu_eol=copy.deepcopy(glu_eol.as_dict())
new_glu_eol['database']=new_db_name
new_glu_eol['code']=new_glu_eol_code
new_glu_eol['name']=new_glu_eol_code
new_glu_eol['exchanges']=[] #empty list of exchanges to add add with for loop below
new_glu_key_eol=(new_db_name,new_glu_eol['code']) #we jsut keep same code and change db for semplicity

to_replace=(new_db_name,'12fabb4b0a9fc33d6f0812186295d724')

for exc in glu_eol.exchanges():        
    data = copy.deepcopy(exc._data)
    data['output'] = new_glu_key_eol
    # Change `input` for production exchanges
    if exc['input'] == exc['output']:
        data['input'] = new_glu_key_eol
    new_glu_eol['exchanges'].append(data)
new_db[new_glu_key_eol]=new_glu_eol
# new_glu_eol

Good, now we have the the dataset `glulam_prod` with the dynamic inventory of the glulam production,  but `glulam_eol` it is still static and we want that also it dynamic. We assume a mean lifetime $k$ for glulam of 50 years. Being lifetime variable we apply a statistical distribution estimate this decay following the approach of [Marland](http://link.springer.com/article/10.1007/s11027-009-9205-6) and parametrizing the gamma function following  [Cherubini](http://onlinelibrary.wiley.com/doi/10.1111/j.1757-1707.2011.01156.x/abstract) i.e. we transform it to a chisquare distribution (already available in temporalis) using parameters $a = k/2$ and $b = 2$ 

In [16]:
from bw2temporalis.dyn_methods.biogenic_carbon import Wood_Decay
glu_decay=Wood_Decay.chi2(emission_year=50,t_horizon=500,tstep=1)

In [17]:
# plt.plot(glu_decay)

Good, now we could modify the `glulam_eol` making its exchanges dynamic, but this would be more complicate and also logically weirder than creating a new dataset, that is actually our Functional Unit, that has input exchange one cubic meter of `glulam_prod` and one m3 of `glulam_eol`, this latter spread over time considering its `glu_decay`. We create manually the FU dataset as below and write the new database.

In [18]:
# del databases['tempo']

In [19]:
code_FU='glulam_FU'
new_db[(new_db_name,code_FU)]={
 'database':new_db_name,
 'name':code_FU,
 'code':code_FU,
 'unit':'cubic meter',
 'exchanges': 
    [{
   'amount': 1.0,
   'input': (new_db_name, new_glu_code),
   'name': new_glu_eol_code,
   'type': 'technosphere',
   'unit': 'cubic meter'
    },{
   'amount': 1.0,
   'input': (new_db_name, new_glu_eol_code),
   'name': new_glu_code,
   'temporal distribution': [(yr,val) for yr,val in enumerate(glu_decay)],
   'type': 'technosphere',
   'unit': 'cubic meter'
    }],
}

tempo = Database("tempo")
tempo.write(new_db)

Writing activities to SQLite3 database:
0%   100%
[#######] | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 06/14/2017 11:01:08
  Finished: 06/14/2017 11:01:08
  Total time elapsed: 00:00:00
  CPU %: 138.00
  Memory %: 1.87


Here we just add a tag to some of the processes that will give grouped results for easier interpretation. See `group` in DynanimcLCA class of Temporalis

In [5]:
for x in [  ('tempo', '41ba0faa3437ad2c2fe52b6b8c00857c'),#'market for sawlog and veneer log, softwood, measured as solid wood under bark' (cubic meter, CH, None)
            ('tempo', 'abb367d014840ef43a947fbd2b75ab5b'),# 'sawing, softwood'
            ('tempo', 'glulam_eol'),
            ('tempo', 'glulam_prod'),
            ]:
    act=get_activity(x)
    act['tempo_group'] = True
    act.save()


Here we show another important thing. As you can see from the database we created when they are registered by default they are dynamic i.e. are evaluated by the graph traversal.

In [20]:
for db in databases.list:
    print(db,'Static: ',databases[db].get('static','False (by default)'))

Agribalyse 1.3 new biosphere Static:  False (by default)
agribalyse 1.3 Static:  True
biosphere3 Static:  False (by default)
ecoinvent 2.2 Static:  True
ecoinvent 3.2 cutoff Static:  True
glu Static:  True
glu_orig Static:  False (by default)
tempo Static:  False (by default)


If you are using a database that does not contain any temporal information you should always set the attribute `static= True` and the flush() to save it as shown below. Despite not obligatory this will sensibly improve your calculation time. And is the same reason why at the beginning of this notebook we suggested to create a new (dynamic) database copying the unit processes needed from ecoinvent rather than create modified copies of the dataset in it.

Good we have created our dynamic database 'tempo' that is linked upstream to several databases (see below). In the next notebook we'll run the analysis and test the performances

In [21]:
tempo.find_graph_dependents()

{'biosphere3', 'ecoinvent 2.2', 'ecoinvent 3.2 cutoff', 'glu', 'tempo'}

In [22]:
databases.list

['Agribalyse 1.3 new biosphere',
 'agribalyse 1.3',
 'biosphere3',
 'ecoinvent 2.2',
 'ecoinvent 3.2 cutoff',
 'glu',
 'glu_orig',
 'tempo']